In [4]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import homm3_battle_server as h3
import threading

In [14]:
class HoMM3_Battle_Env(Env):
    def __init__(self):
        # ai service
        self.tcp_server = h3.TcpServer()
        self.tcp_thread = threading.Thread(
            target=self.tcp_server.start_tcp_server,
            args=('localhost', 9999),
            daemon=True
        )
        self.tcp_thread.start()
        # game
        self.vcmi_thread = threading.Thread(
            target=h3.start_homm3_test_battle,
            args=[True],
            daemon=True
        )
        self.vcmi_thread.start()

        self.action_space = Discrete(4)

        # TODO: update from json
        self.state = 10 + random.randint(-3,3)
        self.stack_length = 10

        # TODO: after init block for step command


    def step(self, action):
        # TODO: block thread
        # это ход одного из игроков (пока не поменяется сторона)
        # пока оставляем на автомате
        # if val = False hold action

        # TODO: update for steps
        # как вариант класть запрос в очередь запросов сервера
        # ответов клиента
        self.stack_length -= 1

        if self.stack_length <= 0:
            done = True
        else:
            done = False

        self.state += random.randint(-1,1)

        reward = 0

        # placeholder for info
        info = {}

        # return step information
        return self.state, reward, done, info

    def render(self):
        # пока делаем headless
        pass

    def reset(self):
        # self.vcmi_thread.kill()
        # self.tcp_server.kill()
        vcmi_killer_thread = threading.Thread(
            target=h3.kill_vcmi,
        )
        vcmi_killer_thread.start()
        tcp_killer_thread = threading.Thread(
            target=self.tcp_server.stop_tcp_server,
        )
        tcp_killer_thread.start()
        tcp_killer_thread.join()
        vcmi_killer_thread.join()
        self.state = 5
        return self.state

In [15]:
env = HoMM3_Battle_Env()

In [16]:
env.reset()

Vcmi killed.
Server stopped.


No matching processes belonging to you were found


5

In [18]:
# TODO: TEST AND DEVELOP!
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Server stopped.
Vcmi killed.


No matching processes belonging to you were found


Episode:1 Score:0
Server stopped.
Vcmi killed.


No matching processes belonging to you were found


Episode:2 Score:0
Server stopped.
Vcmi killed.


No matching processes belonging to you were found


KeyboardInterrupt: 